In [7]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras import layers, models

In [ ]:
df=pd.read_csv("Ankleshwar.csv")
df=df.iloc[:,1:]
df

,Date,pm25
0,04-02-2019 19:00,70.52
1,04-02-2019 20:00,61.39
2,04-02-2019 21:00,59.94
3,04-02-2019 22:00,76.44
4,04-02-2019 23:00,98.54
...,...,...
33531,02-12-2022 22:00,115.37
33532,02-12-2022 23:00,104.06
33533,03-12-2022 00:00,110.59
33534,03-12-2022 01:00,122.87


: 

In [ ]:


# Load your data (replace with actual loading if needed)
# df = pd.read_csv('your_file.csv')  # Example placeholder

# Normalize pm25 column
scaler = MinMaxScaler()
df['pm25_scaled'] = scaler.fit_transform(df[['pm25']])

# Sequence creation function
def create_sequences(data, window=24):
    X, y = [], []
    for i in range(len(data) - window):
        if not np.isnan(data[i + window]):
            seq_x = data[i:i+window]
            if not np.any(np.isnan(seq_x)):
                X.append(seq_x)
                y.append(data[i + window])
    return np.array(X), np.array(y)

# Prepare input and target sequences
pm25_array = df['pm25_scaled'].values
X, y = create_sequences(pm25_array)

# Reshape for GRU model
X = X.reshape((X.shape[0], X.shape[1], 1))  # [samples, timesteps, features]

# biGRU model
model = models.Sequential([
    layers.Bidirectional(layers.GRU(64, return_sequences=False), input_shape=(24, 1)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=25, batch_size=32, verbose=1)

# Imputation for missing values
pm25_filled = df['pm25_scaled'].copy()
window = 24

for i in range(window, len(pm25_filled)):
    if np.isnan(pm25_filled[i]):
        context = pm25_filled[i-window:i]
        if not np.any(np.isnan(context)):
            input_seq = np.array(context).reshape((1, window, 1))  # Fixed here
            prediction = model.predict(input_seq, verbose=0)[0, 0]
            pm25_filled[i] = prediction

# Inverse scale to original range
df['pm25_bigru'] = scaler.inverse_transform(pm25_filled.values.reshape(-1, 1))

C:\Users\Subham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 68s 40ms/step - loss: 6.2175e-04
Epoch 2/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 43s 44ms/step - loss: 1.6253e-04
Epoch 3/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 34s 32ms/step - loss: 1.7656e-04
Epoch 4/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 23s 36ms/step - loss: 1.7543e-04
Epoch 5/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 21s 33ms/step - loss: 1.6596e-04
Epoch 6/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - loss: 1.7549e-04
Epoch 7/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 23s 35ms/step - loss: 1.9943e-04
Epoch 8/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 22s 34ms/step - loss: 1.7384e-04
Epoch 9/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 23s 36ms/step - loss: 1.7151e-04
Epoch 10/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 21s 33ms/step - loss: 1.4777e-04
Epoch 11/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 22s 35ms/step - loss: 2.1773e-04
Epoch 12/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 22s 35ms/step - loss: 1.6529e-04
Epoch 13/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - loss: 2.1402e-04
Epoch 14/25
634/634 ━━━━━━━━━━━━━━━━━━━━ 23s 36

In [ ]:
df.to_csv("biGRU.csv")